In [9]:
import jsonlines
data_1 = []
with jsonlines.open("/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/my_data/with_solar_info/brave/train_test_split/8:2/test_data1.jsonl", mode="r") as file_jsonl:
    for item in file_jsonl.iter():
        data_1.append(item)


In [11]:
# can be classified as true
# the claim is true
# is accurate.(,)

cnt = 0
for i, item in enumerate(data_1):
    if "can be classified as true" in item["query"].lower() and item["response"] == "FALSE.":
        cnt += 1
    if "the claim is true" in item["query"].lower() and item["response"] == "FALSE.":
        cnt += 1
    if "is accurate." in item["query"].lower() and item["response"] == "FALSE.":
        cnt += 1
    if "is accurate," in item["query"].lower() and item["response"] == "FALSE.":
        cnt += 1

cnt


170

In [5]:
# 修改checkpoint路径
ckpt_dir = "/home/hanlv/workspace/code/research/infodemic/LLM/swift/examples/pytorch/llm/output/openchat_3.5/with_solar_info/brave/data1-split=8:2-ratio=1.0/adalora/lr=3.5e-4-20240216-07:50:13/checkpoint-609"

import os
import sys
import json
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

dirs = [".."]
for _dir in dirs:
    if _dir not in sys.path:
        sys.path.append(_dir)

from swift.llm import (
    get_model_tokenizer, get_template, inference
)
from swift.tuners import Swift
from custom import CustomModelType, CustomTemplateType
import evaluation

with open(f"{ckpt_dir}/sft_args.json", "r") as f:
    sft_args = json.load(f)

def get_model_template():
    model_type, template_type = sft_args["model_type"], sft_args["template_type"]
    model, tokenizer = get_model_tokenizer(
        model_type, model_kwargs={'device_map': 'auto'},
        model_dir=sft_args["model_cache_dir"]
    )
    model = Swift.from_pretrained(model, ckpt_dir, inference_mode=True)
    if sft_args["sft_type"] == 'adalora':
        model = model.to(model.dtype)
    model.generation_config.max_new_tokens = 512
    model.generation_config.do_sample = False

    template = get_template(template_type, tokenizer)

    return model, template

model, template = get_model_template()



2024-02-16 15:17:24,726 - modelscope - INFO - PyTorch version 2.1.2 Found.
2024-02-16 15:17:24,728 - modelscope - INFO - Loading ast index from /home/hanlv/.cache/modelscope/ast_indexer
2024-02-16 15:17:24,753 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 dc3cda2b56a86b0242cd145399a14071 and a total number of 953 components indexed
[INFO:swift] Setting torch_dtype: torch.bfloat16
[INFO:swift] model_config: MistralConfig {
  "_name_or_path": "/home/css/models/openchat-3.5-0106",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": fa

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
items = [
{"query": "According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.\n\nCLAIM:\nPublication date: 2020-07-08\nContent: This 1998 photo shows young polio patients in Sierra Leone\n\nPRIOR KNOWLEDGE:\nInformation 1:\nPublication date: 2020-07-28\nTitle: This 1998 photo shows young polio patients in Sierra Leone | Fact ...\nContent:\nA reverse image search on Google found an image showing similarly-dressed children in this photo collection titled “WAR ON POLIO”. A screenshot of Jean-Marc Giboux’s website, taken on July 8, 2020 · The collection was published by French photographer Jean-Marc Giboux, who said he took the photo in the misleading posts in Freetown, the capital of Sierra Leone in 1998.\nThe collection was published by French photographer Jean-Marc Giboux, who said he took the photo in the misleading posts in Freetown, the capital of Sierra Leone in 1998. “I was documenting the polio vaccinations for WHO and these kids [featured in the photo] are polio victims, showing the effects of no vaccination against polio,” Giboux told AFP in a July 7 email. “Since there was no place to get reconstructive surgery in Sierra Leone because of the civil war, they grew up with the consequence of polio, without any vaccination.”\nVaccines have been critical in stopping polio in many parts of the world, but children are still vulnerable to the disease in Sierra Leone.\n“Since there was no place to get reconstructive surgery in Sierra Leone because of the civil war, they grew up with the consequence of polio, without any vaccination.” · The World Health Organization (WHO) also said in a July 7 email that Giboux was on contract for the global health agency when he took the photo. Some of Giboux’s work covering polio for the WHO is available online here.\nInformation 2:\nPublication date: 2022-02-04\nTitle: Fact check: Image shows African children suffering from polio, ...\nContent:\nFrench photographer Jean-Marc Giboux took the photo in 1998 while on a trip to document the effects of polio commissioned by the World Health Organization. Giboux confirmed this to the Associated Press in 2020. “They were kids in a center for children affected with polio, and it was during the Sierra Leone civil war,” he told the news service.\nBased on our research, we rate FALSE the claim that a photo shows African youths disfigured by vaccines. The image in question actually shows the consequences of not having vaccines, as it portrays children affected with polio in 1998 in Sierra Leone who weren't vaccinated for the disease.\nThe image used in this meme has long circulated online as a vivid depiction of the effects of polio. French photographer Jean-Marc Giboux took the photo in 1998 while on a trip to document the effects of polio commissioned by the World Health Organization. Giboux confirmed this to the Associated Press in 2020.\n“They were kids in a center for children affected with polio, and it was during the Sierra Leone civil war,” he told the news service. Giboux’s personal website includes another image of what appears to be many of the same children. Follow us on Facebook! Like our page to get updates throughout the day on our latest debunks · The center Giboux referenced is the Cheshire Home for the Handicapped in Freetown, Sierra Leone’s capital.\nInformation 3:\nPublication date: None\nTitle: Sierra Leone Celebrates Africa’s landmark achievement against ...\nContent:\nMany years ago, when everyday more than 1000 cases of polio were recorded globally, the thought of reaching this stage in the fight against the disease was farfetched. However, today’s event is evidence of the fact that immunization campaigns and strong routine services are important first steps of protection against preventable diseases”, says Evans Liyosi, WHO Representative in Sierra Leone.\nIt has been a people’s agenda and we are proud as a country to have made significant contributions to this great achievement”, say Dr Alpha Tejan Wurie, Minister of Health and Sanitation · Wild poliovirus is the second disease to be eradicated on the African continent after smallpox in 1980. The last case of smallpox was reported in Somalia in 1977. In Sierra Leone, the wild poliovirus is the third disease to be eradicated after smallpox and Yaws.\nIn Sierra Leone, the wild poliovirus is the third disease to be eradicated after smallpox and Yaws. “Today’s milestone has been a long arduous journey. Many years ago, when everyday more than 1000 cases of polio were recorded globally, the thought of reaching this stage in the fight against the disease was farfetched.\nHowever, today’s event is evidence of the fact that immunization campaigns and strong routine services are important first steps of protection against preventable diseases”, says Evans Liyosi, WHO Representative in Sierra Leone. “We feel particularly proud that we are celebrating this day in our · own lifetime. However, we have a long and critical transition ahead of us as the virus is still in circulation in at least two countries in the world. That requires us to continue surveillance and to vaccinate our children against the disease until polio is completely eradicated from every country”. The Sierra Leone government’s commitment has been strong as demonstrated by the establishment of robust national disease surveillance system, and strategic leadership of the Ministry of Health and Sanitation in organizing national and subnational polio immunization campaigns and routine immunization services, some of which were synchronized with other sister countries in the region and supported b\nInformation 4:\nPublication date: 2021-06-23\nTitle: Slideshow: Journey to Sierra Leone for the Debut of a Breakthrough ...\nContent:\nIn Sierra Leone, the first cases were isolated in December 2020 in Kambia, Tonkolili and Western Area Urban districts. The most recent numbers show that 32 cases are now in 7 districts. The Task Force’s Polio Eradication Center has been working with Sierra Leone for the last six months to equip them for the roll out of this new vaccine.\nBut in places where vaccination coverage is low and there is inadequate water and sanitation infrastructure, this excreted vaccine virus can reintroduce poliovirus into the community and, in rare instances, lead to an outbreak of paralytic polio. Since January 2020, there have been more than 900 vaccine-derived polio cases across 24 African countries. In Sierra Leone, the first cases were isolated in December 2020 in Kambia, Tonkolili and Western Area Urban districts.\nThe most recent numbers show that 32 cases are now in 7 districts. The Task Force’s Polio Eradication Center has been working with Sierra Leone for the last six months to equip them for the roll out of this new vaccine. Journey with Task Force Polio Surge Team Senior Epidemiologist Victor Eboh to learn how they do it.\nThe Task Force’s Polio Eradication Center has been working with Sierra Leone for the last six months to equip them for the roll out of this new vaccine. Journey with Task Force Polio Surge Team Senior Epidemiologist Victor Eboh to learn how they do it. A flyer taped up at a health facility in Freetown, Sierra Leone helps build awareness about the importance of getting children vaccinated.\nInformation 5:\nPublication date: None\nTitle: Sierra Leone Celebrates Africa’s landmark achievement against ...\nContent:\nMany years ago, when everyday more than 1000 cases of polio were recorded globally, the thought of reaching this stage in the fight against the disease was farfetched. However, today’s event is evidence of the fact that immunization campaigns and strong routine services are important first steps of protection against preventable diseases”, says Evans Liyosi, WHO Representative in Sierra Leone.\nHowever, today’s event is evidence of the fact that immunization campaigns and strong routine services are important first steps of protection against preventable diseases”, says Evans Liyosi, WHO Representative in Sierra Leone. “We feel particularly proud that we are celebrating this day in our own lifetime. However, we have a long and critical transition ahead of us as the virus is still in circulation in at least two countries in the world. That requires us to continue surveillance and to vaccinate our children against the disease until polio is completely eradicated from every country”. The Sierra Leone government’s commitment has been strong as demonstrated by the establishment of robust national disease surveillance system, and strategic leadership of the Ministry of Health and Sanitation in organizing national and subnational polio immunization campaigns and routine immunization services, some of which were synchronized with other sister countries in the region and supported by \nIt has been a people’s agenda and we are proud as a country to have made significant contributions to this great achievement”, say Dr Alpha Tejan Wurie, Minister of Health and Sanitation · Wild poliovirus is the second disease to be eradicated on the African continent after smallpox in 1980. The last case of smallpox was reported in Somalia in 1977. In Sierra Leone, the wild poliovirus is the third disease to be eradicated after smallpox and Yaws.\nIn Sierra Leone, the wild poliovirus is the third disease to be eradicated after smallpox and Yaws. “Today’s milestone has been a long arduous journey. Many years ago, when everyday more than 1000 cases of polio were recorded globally, the thought of reaching this stage in the fight against the disease was farfetched.\n\nTo classify the claim accurately, we need to analyze the provided information and determine if it supports or contradicts the claim.\n\nClaim: Publication date: 2020-07-08, Content: This 1998 photo shows young polio patients in Sierra Leone.\n\nInformation 1:\n- Confirms that the photo was taken in 1998 by French photographer Jean-Marc Giboux in Freetown, Sierra Leone's capital.\n- Giboux stated that the children in the photo are polio victims and were not vaccinated due to the civil war in Sierra Leone.\n- The World Health Organization (WHO) also confirmed that Giboux was working for them when he took the photo.\n\nInformation 2:\n- Reiterates that the photo was taken by Jean-Marc Giboux in 1998 while documenting the effects of polio in Sierra Leone.\n- The image shows the consequences of not having vaccines, as it depicts children affected by polio who were not vaccinated.\n\nInformation 3:\n- This information does not directly relate to the claim but provides context about the eradication of polio in Africa and Sierra Leone.\n\nInformation 4:\n- This information is about the introduction of a new vaccine in Sierra Leone due to vaccine-derived polio cases. It does not directly relate to the claim.\n\nInformation 5:\n- Similar to Information 3, this information provides context about the eradication of polio in Africa and Sierra Leone.\n\nBased on the analysis of Information 1 and Information 2, they both confirm that the photo was taken in 1998 in Sierra Leone and shows young polio patients. Therefore, the claim can be classified as TRUE.", "response": "FALSE."}
	
]
_id = 0

query = items[_id]["query"]
# query = 'what is 1+1?'
print(query + '\n')
response, history = inference(
    model, template, query, 
    verbose=False, prompt_prefix='', output_prefix=''
)

print(f'response: {response} label: {items[_id]["response"]}')


According to the CLAIM and the PRIOR KNOWLEDGE, please classify the CLAIM as TRUE or FALSE. If the content described by the CLAIM is correct, then classify it as TRUE; if the content described by the CLAIM is incorrect, then classify it as FALSE.

CLAIM:
Publication date: 2020-07-08
Content: This 1998 photo shows young polio patients in Sierra Leone

PRIOR KNOWLEDGE:
Information 1:
Publication date: 2020-07-28
Title: This 1998 photo shows young polio patients in Sierra Leone | Fact ...
Content:
A reverse image search on Google found an image showing similarly-dressed children in this photo collection titled “WAR ON POLIO”. A screenshot of Jean-Marc Giboux’s website, taken on July 8, 2020 · The collection was published by French photographer Jean-Marc Giboux, who said he took the photo in the misleading posts in Freetown, the capital of Sierra Leone in 1998.
The collection was published by French photographer Jean-Marc Giboux, who said he took the photo in the misleading posts in Freeto